# 仅使用random与math库实现神经网络

In [1]:
import random
import math

## 神经元类

In [2]:
class Neuron:
    def __init__(self, bias):
        self.bias = bias
        self.weights = []

    # 根据输入，通过激活函数，计算神经元的输出
    def calculate_output(self, inputs):
        self.inputs = inputs
        self.output = self.sigmoid(self.calculate_total_net_input())
        return self.output

    # 通过上一层神经元以及权重以及偏置项，得到该神经元的值
    def calculate_total_net_input(self):
        total = 0
        for i in range(len(self.inputs)):
            total += self.inputs[i] * self.weights[i]
        return total + self.bias

    # 激活函数sigmoid
    def sigmoid(self, total_net_input):
        return 1 / (1 + math.exp(-total_net_input))

    # 神经元的输出偏导 * 神经元的输入项偏导 = 神经元的自身的偏导数
    def calculate_pd_error_wrt_total_net_input(self, target_output):
        return self.calculate_pd_error_wrt_output(target_output) * \
               self.calculate_pd_total_net_input_wrt_input()

    # 每一个神经元的误差是由平方差公式计算的
    def calculate_error(self, target_output):
        return 0.5 * (target_output - self.output) ** 2

    # 对神经元的输出求偏导，即对损失函数：平方差公式求偏导
    def calculate_pd_error_wrt_output(self, target_output):
        return -(target_output - self.output)

    # 对神经元的输入项求偏导，即对激活函数：Sigmoid求偏导，Sigmoid的导数为Sigmoid * (1 - Sigmoid)
    def calculate_pd_total_net_input_wrt_input(self):
        return self.output * (1 - self.output)

    # 对神经元的某权重项求偏导，即与权重对应索引的上一个神经元本身的值
    def calculate_pd_total_net_input_wrt_weight(self, index):
        return self.inputs[index]

## 神经网络层类

In [3]:
class NeuronLayer:
    def __init__(self, num_neurons, bias):
        # 同一层的神经元共享一个截距项b
        self.bias = bias if bias else random.random()
        self.neurons = []
        for i in range(num_neurons):
            self.neurons.append(Neuron(self.bias))

    # 输出该层所有神经元的权重以及偏置项的值
    def inspect(self):
        print('Neurons:', len(self.neurons))
        for n in range(len(self.neurons)):
            print(' Neuron', n)
            for w in range(len(self.neurons[n].weights)):
                print('  Weight:', self.neurons[n].weights[w])
            print('  Bias:', self.bias)

    # 前馈算法，非常简单
    def feed_forward(self, inputs):
        outputs = []
        for neuron in self.neurons:
            outputs.append(neuron.calculate_output(inputs))
        return outputs

    # 得到该神经层所有神经元，经过激活函数计算得到的输出项
    def get_outputs(self):
        outputs = []
        for neuron in self.neurons:
            outputs.append(neuron.output)
        return outputs

## 神经网络类

In [4]:
#   参数解释：
#   "pd_" ：偏导的前缀
#   "d_" ：导数的前缀
#   "w_ho" ：隐含层到输出层的权重系数索引
#   "w_ih" ：输入层到隐含层的权重系数的索引

class NeuralNetwork:
    LEARNING_RATE = 0.5
    def __init__(self,
                 num_inputs,
                 num_hidden,
                 num_outputs,
                 hidden_layer_weights = None,
                 hidden_layer_bias = None,
                 output_layer_weights = None,
                 output_layer_bias = None):
        self.num_inputs = num_inputs

        self.hidden_layer = NeuronLayer(num_hidden, hidden_layer_bias)
        self.output_layer = NeuronLayer(num_outputs, output_layer_bias)

        self.init_weights_from_inputs_to_hidden_layer_neurons(hidden_layer_weights)
        self.init_weights_from_hidden_layer_neurons_to_output_layer_neurons(output_layer_weights)

    def init_weights_from_inputs_to_hidden_layer_neurons(self, hidden_layer_weights):
        weight_num = 0
        for h in range(len(self.hidden_layer.neurons)):
            for i in range(self.num_inputs):
                if not hidden_layer_weights:
                    self.hidden_layer.neurons[h].weights.append(random.random())
                else:
                    self.hidden_layer.neurons[h].weights.append(hidden_layer_weights[weight_num])
                weight_num += 1

    def init_weights_from_hidden_layer_neurons_to_output_layer_neurons(self, output_layer_weights):
        weight_num = 0
        for o in range(len(self.output_layer.neurons)):
            for h in range(len(self.hidden_layer.neurons)):
                if not output_layer_weights:
                    self.output_layer.neurons[o].weights.append(random.random())
                else:
                    self.output_layer.neurons[o].weights.append(output_layer_weights[weight_num])
                weight_num += 1

    def inspect(self):
        print('------')
        print('* Inputs: {}'.format(self.num_inputs))
        print('------')
        print('Hidden Layer')
        self.hidden_layer.inspect()
        print('------')
        print('* Output Layer')
        self.output_layer.inspect()
        print('------')

    def feed_forward(self, inputs):
        hidden_layer_outputs = self.hidden_layer.feed_forward(inputs)
        return self.output_layer.feed_forward(hidden_layer_outputs)

    # 训练方法，先实现前馈更新各个神经元的值，并通过链式求导法则，反向传播，
    # 从而更新神经网络中的各级权重
    def train(self, training_inputs, training_outputs):
        self.feed_forward(training_inputs)

        # 1. 输出神经元的值
        pd_errors_wrt_output_neuron_total_net_input = [0] * len(self.output_layer.neurons)
        for o in range(len(self.output_layer.neurons)):
            # ∂E/∂zⱼ
            pd_errors_wrt_output_neuron_total_net_input[o] = \
                self.output_layer.neurons[o].calculate_pd_error_wrt_total_net_input(training_outputs[o])

        # 2. 隐含层神经元的值
        pd_errors_wrt_hidden_neuron_total_net_input = [0] * len(self.hidden_layer.neurons)
        for h in range(len(self.hidden_layer.neurons)):
            # dE/dyⱼ = Σ ∂E/∂zⱼ * ∂z/∂yⱼ = Σ ∂E/∂zⱼ * wᵢⱼ
            d_error_wrt_hidden_neuron_output = 0
            for o in range(len(self.output_layer.neurons)):
                d_error_wrt_hidden_neuron_output += \
                    pd_errors_wrt_output_neuron_total_net_input[o] * self.output_layer.neurons[o].weights[h]

            # ∂E/∂zⱼ = dE/dyⱼ * ∂zⱼ/∂
            pd_errors_wrt_hidden_neuron_total_net_input[h] = \
                d_error_wrt_hidden_neuron_output * self.hidden_layer.neurons[h].calculate_pd_total_net_input_wrt_input()

        # 3. 更新输出层权重系数
        for o in range(len(self.output_layer.neurons)):
            for w_ho in range(len(self.output_layer.neurons[o].weights)):
                # ∂Eⱼ/∂wᵢⱼ = ∂E/∂zⱼ * ∂zⱼ/∂wᵢⱼ
                pd_error_wrt_weight = \
                    pd_errors_wrt_output_neuron_total_net_input[o] * \
                    self.output_layer.neurons[o].calculate_pd_total_net_input_wrt_weight(w_ho)

                # Δw = α * ∂Eⱼ/∂wᵢ
                self.output_layer.neurons[o].weights[w_ho] -= self.LEARNING_RATE * pd_error_wrt_weight

        # 4. 更新隐含层的权重系数
        for h in range(len(self.hidden_layer.neurons)):
            for w_ih in range(len(self.hidden_layer.neurons[h].weights)):
                # ∂Eⱼ/∂wᵢ = ∂E/∂zⱼ * ∂zⱼ/∂wᵢ
                pd_error_wrt_weight = \
                    pd_errors_wrt_hidden_neuron_total_net_input[h] * \
                    self.hidden_layer.neurons[h].calculate_pd_total_net_input_wrt_weight(w_ih)

                # Δw = α * ∂Eⱼ/∂wᵢ
                self.hidden_layer.neurons[h].weights[w_ih] -= self.LEARNING_RATE * pd_error_wrt_weight

    # 得到每次训练之后的输出以及总体误差值（为输出层所有神经元输出的误差和）
    def calculate_total_error(self, training_sets):
        total_error = 0
        for t in range(len(training_sets)):
            training_inputs, training_outputs = training_sets[t]
            outputs = self.feed_forward(training_inputs)
            print('outputs: ', outputs)
            for o in range(len(training_outputs)):
                total_error += self.output_layer.neurons[o].calculate_error(training_outputs[o])
        return total_error

## 使用神经网络

In [5]:
nn = NeuralNetwork(num_inputs=2,
                   num_hidden=2,
                   num_outputs=2,
                   hidden_layer_weights=[0.15, 0.2, 0.25, 0.3],
                   hidden_layer_bias=0.35,
                   output_layer_weights=[0.4, 0.45, 0.5, 0.55],
                   output_layer_bias=0.6)
for i in range(10000):
    nn.train([0.05, 0.1], [0.01, 0.09])
    print(i, 'error:', round(nn.calculate_total_error([[[0.05, 0.1], [0.01, 0.09]]]), 9))

outputs:  [0.7420836778818768, 0.76538496769113]
0 error: 0.496045683
outputs:  [0.7324693882960343, 0.7575794693956527]
1 error: 0.483812182
outputs:  [0.7225297921452635, 0.7495134856808008]
2 error: 0.471328371
outputs:  [0.7122757898715133, 0.7411903012927673]
3 error: 0.458620047
outputs:  [0.7017217420964527, 0.7326151116647087]
4 error: 0.445716575
outputs:  [0.6908855425609495, 0.7237951412577168]
5 error: 0.432650702
outputs:  [0.6797885992316263, 0.7147397319569513]
6 error: 0.41945825
outputs:  [0.6684557132441958, 0.7054603946118289]
7 error: 0.406177712
outputs:  [0.6569148500663811, 0.6959708174507939]
8 error: 0.392849727
outputs:  [0.6451968030889439, 0.6862868262350071]
9 error: 0.379516479
outputs:  [0.6333347563597299, 0.6764262926394644]
10 error: 0.366221008
outputs:  [0.6213637597789656, 0.6664089894134662]
11 error: 0.353006485
outputs:  [0.6093201361069102, 0.6562563932709667]
12 error: 0.339915464
outputs:  [0.5972408439396579, 0.6459914390417505]
13 error: 0.3

outputs:  [0.1118999072902351, 0.15159529394756271]
193 error: 0.007088786
outputs:  [0.11153211182146315, 0.15122208700951248]
194 error: 0.007028457
outputs:  [0.1111677162901318, 0.1508525983482153]
195 error: 0.006968973
outputs:  [0.11080667074446243, 0.1504867721226063]
196 error: 0.006910317
outputs:  [0.11044892622723827, 0.1501245536189424]
197 error: 0.006852474
outputs:  [0.11009443475099755, 0.14976588922241688]
198 error: 0.006795429
outputs:  [0.10974314927396635, 0.1494107263896246]
199 error: 0.006739165
outputs:  [0.10939502367670506, 0.14905901362184878]
200 error: 0.006683669
outputs:  [0.10905001273944447, 0.14871070043914122]
201 error: 0.006628926
outputs:  [0.10870807212008733, 0.1483657373551685]
202 error: 0.006574921
outputs:  [0.10836915833285371, 0.1480240758527978]
203 error: 0.006521642
outputs:  [0.10803322872754687, 0.14768566836039806]
204 error: 0.006469075
outputs:  [0.1077002414694201, 0.14735046822883152]
205 error: 0.006417207
outputs:  [0.10737015

outputs:  [0.07286468092777106, 0.11420410746393213]
393 error: 0.002268903
outputs:  [0.07275877181591096, 0.11411180922148054]
394 error: 0.002260021
outputs:  [0.07265332106793738, 0.11401998583799001]
395 error: 0.002251199
outputs:  [0.07254832547030723, 0.11392863386676313]
396 error: 0.002242436
outputs:  [0.072443781840463, 0.1138377498948382]
397 error: 0.002233732
outputs:  [0.07233968702645373, 0.11374733054257213]
398 error: 0.002225086
outputs:  [0.07223603790656138, 0.11365737246322834]
399 error: 0.002216498
outputs:  [0.07213283138893291, 0.11356787234257204]
400 error: 0.002207967
outputs:  [0.07203006441121798, 0.11347882689847083]
401 error: 0.002199492
outputs:  [0.07192773394021136, 0.11339023288050107]
402 error: 0.002191074
outputs:  [0.07182583697150127, 0.11330208706956055]
403 error: 0.002182711
outputs:  [0.07172437052912249, 0.11321438627748683]
404 error: 0.002174403
outputs:  [0.0716233316652143, 0.11312712734668078]
405 error: 0.00216615
outputs:  [0.0715

outputs:  [0.05793884576964233, 0.10213673761475699]
593 error: 0.001222717
outputs:  [0.05788590265816123, 0.10209817078768356]
594 error: 0.001219713
outputs:  [0.05783310790852584, 0.10205974948438315]
595 error: 0.001216722
outputs:  [0.057780460843191224, 0.10202147299522411]
596 error: 0.001213744
outputs:  [0.05772796078888643, 0.1019833406151223]
597 error: 0.001210779
outputs:  [0.057675607076580294, 0.10194535164350359]
598 error: 0.001207827
outputs:  [0.0576233990414475, 0.10190750538426802]
599 error: 0.001204888
outputs:  [0.057571336022834714, 0.10186980114575298]
600 error: 0.001201962
outputs:  [0.05751941736422745, 0.1018322382406979]
601 error: 0.001199048
outputs:  [0.05746764241321691, 0.10179481598620839]
602 error: 0.001196147
outputs:  [0.05741601052146728, 0.10175753370372106]
603 error: 0.001193259
outputs:  [0.057364521044683474, 0.10172039071896899]
604 error: 0.001190383
outputs:  [0.05731317334257905, 0.10168338636194721]
605 error: 0.001187519
outputs:  [

outputs:  [0.04962308052848483, 0.09661046809404385]
793 error: 0.000806843
outputs:  [0.0495903367104172, 0.09659113724703594]
794 error: 0.000805419
outputs:  [0.04955766056834508, 0.09657186808982864]
795 error: 0.000803999
outputs:  [0.049525051873610136, 0.09655266038763079]
796 error: 0.000802584
outputs:  [0.0494925103986236, 0.09653351390677412]
797 error: 0.000801173
outputs:  [0.049460035916859706, 0.09651442841470625]
798 error: 0.000799766
outputs:  [0.04942762820284946, 0.09649540367998448]
799 error: 0.000798364
outputs:  [0.0493952870321744, 0.09647643947226858]
800 error: 0.000796966
outputs:  [0.04936301218146003, 0.0964575355623146]
801 error: 0.000795573
outputs:  [0.0493308034283701, 0.0964386917219679]
802 error: 0.000794184
outputs:  [0.049298660551600156, 0.09641990772415714]
803 error: 0.0007928
outputs:  [0.04926658333087152, 0.09640118334288744]
804 error: 0.00079142
outputs:  [0.04923457154692512, 0.0963825183532342]
805 error: 0.000790044
outputs:  [0.049202

...

9188 error: 2.1187e-05
outputs:  [0.01650879968910698, 0.08997585250804281]
9189 error: 2.1183e-05
outputs:  [0.016508149360285527, 0.08997585581491004]
9190 error: 2.1178e-05
outputs:  [0.0165074991462166, 0.08997585912100617]
9191 error: 2.1174e-05
outputs:  [0.016506849046868263, 0.08997586242633161]
9192 error: 2.117e-05
outputs:  [0.01650619906220847, 0.08997586573088662]
9193 error: 2.1166e-05
outputs:  [0.016505549192205253, 0.08997586903467136]
9194 error: 2.1161e-05
outputs:  [0.016504899436826644, 0.08997587233768621]
9195 error: 2.1157e-05
outputs:  [0.016504249796040652, 0.08997587563993137]
9196 error: 2.1153e-05
outputs:  [0.01650360026981536, 0.08997587894140716]
9197 error: 2.1149e-05
outputs:  [0.016502950858118832, 0.08997588224211377]
9198 error: 2.1144e-05
outputs:  [0.016502301560919107, 0.0899758855420515]
9199 error: 2.114e-05
outputs:  [0.016501652378184323, 0.08997588884122065]
9200 error: 2.1136e-05
outputs:  [0.016501003309882533, 0.08997589213962144]
9201 er

9388 error: 2.0363e-05
outputs:  [0.016380976245375725, 0.0899764988804673]
9389 error: 2.0359e-05
outputs:  [0.016380348245963072, 0.0899765020382756]
9390 error: 2.0355e-05
outputs:  [0.016379720355139942, 0.08997650519536379]
9391 error: 2.0351e-05
outputs:  [0.016379092572876643, 0.08997650835173211]
9392 error: 2.0347e-05
outputs:  [0.016378464899143567, 0.08997651150738079]
9393 error: 2.0343e-05
outputs:  [0.016377837333911117, 0.08997651466231014]
9394 error: 2.0339e-05
outputs:  [0.016377209877149627, 0.0899765178165203]
9395 error: 2.0335e-05
outputs:  [0.016376582528829533, 0.08997652097001162]
9396 error: 2.0331e-05
outputs:  [0.016375955288921235, 0.08997652412278422]
9397 error: 2.0327e-05
outputs:  [0.016375328157395137, 0.08997652727483847]
9398 error: 2.0323e-05
outputs:  [0.01637470113422168, 0.08997653042617454]
9399 error: 2.0319e-05
outputs:  [0.01637407421937133, 0.08997653357679272]
9400 error: 2.0315e-05
outputs:  [0.016373447412814518, 0.08997653672669317]
9401

9588 error: 1.9582e-05
outputs:  [0.01625749906980068, 0.0899771164242858]
9589 error: 1.9578e-05
outputs:  [0.016256892212743984, 0.08997711944276941]
9590 error: 1.9575e-05
outputs:  [0.01625628545856425, 0.0899771224605794]
9591 error: 1.9571e-05
outputs:  [0.016255678807234022, 0.08997712547771604]
9592 error: 1.9567e-05
outputs:  [0.016255072258725776, 0.08997712849417949]
9593 error: 1.9563e-05
outputs:  [0.01625446581301204, 0.08997713150997004]
9594 error: 1.9559e-05
outputs:  [0.01625385947006533, 0.08997713452508792]
9595 error: 1.9556e-05
outputs:  [0.016253253229858218, 0.08997713753953326]
9596 error: 1.9552e-05
outputs:  [0.01625264709236321, 0.08997714055330633]
9597 error: 1.9548e-05
outputs:  [0.016252041057552895, 0.08997714356640742]
9598 error: 1.9544e-05
outputs:  [0.016251435125399845, 0.0899771465788366]
9599 error: 1.954e-05
outputs:  [0.016250829295876606, 0.08997714959059422]
9600 error: 1.9537e-05
outputs:  [0.01625022356895581, 0.08997715260168043]
9601 erro

9788 error: 1.8842e-05
outputs:  [0.01613813943788938, 0.08997770700220764]
9789 error: 1.8839e-05
outputs:  [0.01613755262204724, 0.0899777098902707]
9790 error: 1.8835e-05
outputs:  [0.01613696590377873, 0.0899777127777028]
9791 error: 1.8831e-05
outputs:  [0.016136379283058312, 0.08997771566450402]
9792 error: 1.8828e-05
outputs:  [0.016135792759860438, 0.08997771855067473]
9793 error: 1.8824e-05
outputs:  [0.01613520633415958, 0.08997772143621502]
9794 error: 1.8821e-05
outputs:  [0.016134620005930213, 0.08997772432112508]
9795 error: 1.8817e-05
outputs:  [0.016134033775146796, 0.08997772720540512]
9796 error: 1.8813e-05
outputs:  [0.01613344764178386, 0.08997773008905538]
9797 error: 1.881e-05
outputs:  [0.0161328616058159, 0.08997773297207609]
9798 error: 1.8806e-05
outputs:  [0.016132275667217404, 0.08997773585446739]
9799 error: 1.8803e-05
outputs:  [0.016131689825962917, 0.08997773873622947]
9800 error: 1.8799e-05
outputs:  [0.016131104082026986, 0.0899777416173627]
9801 error

9988 error: 1.814e-05
outputs:  [0.016022685015103642, 0.08997827231975955]
9989 error: 1.8137e-05
outputs:  [0.01602211721743507, 0.08997827508556301]
9990 error: 1.8133e-05
outputs:  [0.016021549512409125, 0.08997827785077456]
9991 error: 1.813e-05
outputs:  [0.016020981900002056, 0.08997828061539434]
9992 error: 1.8126e-05
outputs:  [0.016020414380190084, 0.08997828337942257]
9993 error: 1.8123e-05
outputs:  [0.016019846952949442, 0.08997828614285935]
9994 error: 1.812e-05
outputs:  [0.016019279618256402, 0.08997828890570497]
9995 error: 1.8116e-05
outputs:  [0.016018712376087235, 0.08997829166795959]
9996 error: 1.8113e-05
outputs:  [0.01601814522641815, 0.08997829442962334]
9997 error: 1.8109e-05
outputs:  [0.016017578169225475, 0.08997829719069655]
9998 error: 1.8106e-05
outputs:  [0.016017011204485475, 0.0899782999511793]
9999 error: 1.8102e-05
